In [2]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns


/tmp/ipykernel_302561/4141160823.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
train_data = pd.read_csv('data/train.csv.zip', compression='zip')
test_data = pd.read_csv('data/test.csv.zip', compression='zip')
sample_submission_data = pd.read_csv("data/sample_submission.csv")

label_encoder = LabelEncoder()
train_data['Target'] = label_encoder.fit_transform(train_data['Target'])
train_data = train_data.drop(['id'], axis=1)
test_data = test_data.drop(['id'], axis=1)

train_data.drop_duplicates(inplace = True)
train_data.dropna(inplace = True)


In [4]:
train_data.drop(["Previous qualification", "Nacionality", "Mother's qualification", "Father's qualification", "Mother's occupation", 
         "Father's occupation", "Educational special needs", "International", "Curricular units 1st sem (credited)", 
         "Curricular units 1st sem (without evaluations)", "Curricular units 2nd sem (credited)", "Curricular units 2nd sem (grade)", 
         "Curricular units 2nd sem (without evaluations)", "Unemployment rate", "Inflation rate"], axis=1, inplace=True)
test_data.drop(["Previous qualification", "Nacionality", "Mother's qualification", "Father's qualification", "Mother's occupation", 
         "Father's occupation", "Educational special needs", "International", "Curricular units 1st sem (credited)", 
         "Curricular units 1st sem (without evaluations)", "Curricular units 2nd sem (credited)", "Curricular units 2nd sem (grade)", 
         "Curricular units 2nd sem (without evaluations)", "Unemployment rate", "Inflation rate"], axis=1, inplace=True)

In [5]:
data_train_targets = pd.DataFrame(train_data["Target"])
data_train_features = train_data.drop(columns = ["Target"])
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# using pca to extract the features


pca = PCA(n_components=10)
scaler = StandardScaler()

pipeline = Pipeline([('scaler', scaler), ('pca', pca)])
X_scaled_pca = pipeline.fit_transform(data_train_features)



In [6]:
X_train,X_test,y_train,y_test = train_test_split( data_train_features, data_train_targets, test_size=0.10)
y_train=y_train.values.ravel()

In [33]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
}

# Create the model
model = GradientBoostingClassifier()

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

# Train the model with the best parameters
best_model = GradientBoostingClassifier(**best_params)
best_model.fit(X_train, y_train)


KeyboardInterrupt: 

In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
y_pred=best_model.predict(X_test)
print('accuracy_score:', accuracy_score(y_test, y_pred))

accuracy_score: 0.8273653946680606


In [15]:
y_test_pred = best_model.predict(test_data)
# Create the submission DataFrame
submission = pd.DataFrame({
    'id': sample_submission_data['id'],  # Ensure the 'id' column matches the sample submission
    'Target': label_encoder.inverse_transform(y_test_pred)  # Inverse transform if necessary
})

# Save the submission file
submission.to_csv('submission_gradient_boosting.csv', index=False)

print("Submission file created: submission.csv")

Submission file created: submission.csv
